# Machine Learning for Cryptocurrency Trading
## Section 1: Cleaning Raw Data


In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

# Viewing options
pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000

# from course materials in ~/ml4t
import ml4t

In [2]:
ml4t.TradeData()

TypeError: __init__() missing 1 required positional argument: 'df_s'

In [ ]:
import ml4t

In [ ]:
trade_data.TradeData()

In [ ]:
from ml4t import TradeData as td

In [ ]:
ml4t.TradeData.TradeData()